# Random numbers and reproducibility

Random and stochastic dynamics are an essential part of most agent-based models.
In Python, the built-in library `random` provides a range of useful tools.

In [1]:
import agentpy as ap
import random

To illustrate, let us build a model that generates a list of ten pseudo-random numbers:

In [2]:
class RandomModel(ap.Model):
    
    def setup(self):
        self.random_numbers = [random.randint(0, 9) for _ in range(10)]
        print(f"Model {self.p.n} generated the numbers {self.random_numbers}")

Now if we run this model, we will get different numbers every time:

In [3]:
for i in range(2):
    parameters = {'steps':0, 'n':i}
    model = RandomModel(parameters)
    results = model.run(display=False)

Model 0 generated the numbers [2, 7, 7, 4, 2, 6, 4, 4, 1, 3]
Model 1 generated the numbers [8, 6, 8, 6, 2, 6, 7, 9, 3, 9]


If we want the results to be reproducible (meaning that we want the same random numbers to be generated in every run),
we can define a parameter `seed` that will be used automatically to set a random seed at the beginning of each run:

In [4]:
for i in range(2):
    parameters = {'seed':1, 'steps':0, 'n':i}
    model = RandomModel(parameters)
    model.run(display=False)

Model 0 generated the numbers [2, 9, 1, 4, 1, 7, 7, 7, 6, 3]
Model 1 generated the numbers [2, 9, 1, 4, 1, 7, 7, 7, 6, 3]


We can further create seperate random number generators for each object,
with their seeds still being reproducible between runs.

In [5]:
class RandomAgent(ap.Agent):
    
    def setup(self):
        self.random = random.Random(model.random.getrandbits(128))
        self.random_numbers = [self.random.randint(0, 9) for _ in range(10)]
        print(f"{self} has the numbers {self.random_numbers}")
        
class RandomModel(ap.Model):
    
    def setup(self):
        self.random = random.Random(self.p.seed)
        print(f"Model {self.p.n}:")
        self.add_agents(2, RandomAgent)
        print("")

for i in range(2):
    parameters = {'seed': 1, 'steps':0, 'n':i}
    model = RandomModel2(parameters)
    results = model.run(display=False)

Model 0:
RandomAgent (Obj 1) has the numbers [8, 7, 0, 1, 2, 3, 9, 4, 5, 0]
RandomAgent (Obj 2) has the numbers [8, 1, 4, 6, 6, 3, 4, 3, 5, 1]

Model 1:
RandomAgent (Obj 1) has the numbers [8, 7, 0, 1, 2, 3, 9, 4, 5, 0]
RandomAgent (Obj 2) has the numbers [8, 1, 4, 6, 6, 3, 4, 3, 5, 1]



Alternatively, we could also have each agent start from the same seed:

In [6]:
class RandomAgent2(ap.Agent):
    
    def setup(self):
        self.random = random.Random(self.p.seed)
        self.random_numbers = [self.random.randint(0, 9) for _ in range(10)]
        print(f"{self} has the numbers {self.random_numbers}")
        
class RandomModel2(ap.Model):
    
    def setup(self):
        self.random = random.Random(self.p.seed)
        print(f"Model {self.p.n}:")
        self.add_agents(2, RandomAgent2)
        print("")

for i in range(2):
    parameters = {'seed': 1, 'steps':0, 'n':i}
    model = RandomModel2(parameters)
    results = model.run(display=False)

Model 0:
RandomAgent2 (Obj 1) has the numbers [2, 9, 1, 4, 1, 7, 7, 7, 6, 3]
RandomAgent2 (Obj 2) has the numbers [2, 9, 1, 4, 1, 7, 7, 7, 6, 3]

Model 1:
RandomAgent2 (Obj 1) has the numbers [2, 9, 1, 4, 1, 7, 7, 7, 6, 3]
RandomAgent2 (Obj 2) has the numbers [2, 9, 1, 4, 1, 7, 7, 7, 6, 3]



Final notes:

- Random number generators can also be passed to the methods `AgentList.shuffle()` and `AgentList.random()`.
- The numbers in this document are pseudo-random, not truly random (more info below).

Further information:

- Random number generation in Python: https://realpython.com/python-random/
- Pseudo-random numbers: https://en.wikipedia.org/wiki/Pseudorandom_number_generator
- What is random: https://www.youtube.com/watch?v=9rIy0xY99a0